In [2]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%run -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 49.8 MB/s eta 0:00:0000:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [83]:
from pathlib import Path

import pandas as pd
import spacy

f = Path(f"./tlg0525.tlg001.perseus-eng2.pickle")

def tokenize(df: pd.DataFrame):
    model = "en_core_web_sm"

    nlp = spacy.load(model, disable=["ner"])

    df["tokens"] = df["unannotated_strings"].apply(nlp.tokenizer)

    raw_texts = [t for t in df["unannotated_strings"]]
    annotated_texts = nlp.pipe(raw_texts, batch_size=100)

    df["nlp_docs"] = list(annotated_texts)

    return df

df = tokenize(pd.read_pickle(f))

# Bug 1

Why is the function `expected_frequency_of_collocation(df: pd.DataFrame, node: str, collocate: str, window_size: int = 1)` returning `0.0` no matter what we pass to it?

In [35]:
from collections import Counter

def expected_frequency_of_collocation(df: pd.DataFrame, node: str, collocate: str, window_size: int = 1):
    """
    `node` and `collocate` should be the string representations
    of the associated lemmata
    """

    lemmata = [t for t in df['nlp_docs'].explode()]
    counter = Counter(lemmata)
    node_count = counter[node]
    collocate_count = counter[collocate]

    return (node_count * collocate_count * window_size) / len(lemmata)

expected_freq_the_mainlaind = expected_frequency_of_collocation(df, "mainland", "the")

expected_freq_the_mainlaind

0.0

# Bug 1 analysis and response

I believe this is because lemmata is not properly defined. That is, I think the code should be: 
lemmata = [t.lemma_ for t in df['nlp_docs].explode() 

I also changed the / len (lemmata) to / len(df['tokens].explode()), since I believe we need to divide by the number of tokens--not the number of lemmata--in the corpus in order to get the expected collocation frequency. 

This got me a result of 3.284, although I am not confident this is correct either. 

In [87]:
from collections import Counter

def expected_frequency_of_collocation(df: pd.DataFrame, node: str, collocate: str, window_size: int = 1):
    """
    `node` and `collocate` should be the string representations
    of the associated lemmata
    """

    lemmata = [t.lemma_ for t in df['nlp_docs'].explode()] ## changed this line
    counter = Counter(lemmata)
    node_count = counter[node]
    collocate_count = counter[collocate]

    return (node_count * collocate_count * window_size) / len(df['tokens'].explode()) # I changed this 


expected_freq_the_mainland = expected_frequency_of_collocation(df, "mainland", "the")

expected_freq_the_mainland


3.2848751741271176

In [90]:
lemmata = [t.lemma_ for t in df['nlp_docs'].explode()] ## changed this line
counter = Counter(lemmata)
node_count = counter[node]
collocate_count = counter[collocate]

counter['the']

26932

# Bug 2

Why does the code below not give us the number of occurrences of the token "the"?

In [63]:
observed_freq_the = df['tokens'].str.find('the')

observed_freq_the

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
3165   NaN
3166   NaN
3167   NaN
3168   NaN
3169   NaN
Name: tokens, Length: 3170, dtype: float64

# Bug 2 analysis and response

In [97]:
lemmata = [t.lemma_ for t in df['nlp_docs'].explode()] 
counter = Counter(lemmata)
observed_freq_the = counter['the']

observed_freq_the
## 26,932 occurences of 'the'

26932

The original code didn't work because .find() is not a count function. I don't understand the approach original code was taking, so my "fix" may not be what you are looking for. But instead of tweaking it, I tried a different method to get the frequency using the Counter function to count the occurences of the lemma 'the.' 

# Bug 3

How is `o11` greater than `r1`, and why is our `o12` negative?

In [122]:
node = "mainland"
collocate = "the"

def count_ngram_collocations(x, w1, w2, l_size: int = 1, r_size: int = 1):
    lemmata = [t.lemma_ for t in x]

    # the right-hand side of a slice in Python is exclusive, so we add 1 to make sure
    # we're actually getting one element to the right

    chunked_lemmata = [lemmata[i - l_size:i + r_size + 1] for i in range(0, len(lemmata))]
    
    cooccurrences = [1 for l in chunked_lemmata if w1 in l and w2 in l]

    return sum(cooccurrences)

r1 = len([t for t in df["nlp_docs"].explode() if t.lemma_ == node])
r2 = len([t for t in df["nlp_docs"].explode() if t.lemma_ != node])
c1 = len([t for t in df["nlp_docs"].explode() if t.lemma_ == collocate])
c2 = len([t for t in df["nlp_docs"].explode() if t.lemma_ != collocate])

o11 = (df["nlp_docs"].apply(count_ngram_collocations, args=(node, collocate))).sum()
o12 = r1 - o11
o21 = c1 - o11

print(f"r1: {r1}, r2: {r2}, c1: {c1}, c2: {c2}, o11: {o11}, o12: {o12}, o21: {o21}")

r1: 38, r2: 311516, c1: 26932, c2: 284622, o11: 63, o12: -25, o21: 26869


# Bug 3 analysis and response

In [139]:
def count_ngram_collocations(x, w1, w2, l_size: int = 1, r_size: int = 1):
    lemmata = [t.lemma_ for t in x]

    indexes = [i for i, lemma in enumerate(lemmata) if lemma == w1]

    cooccurrences = 0

    for i in indexes:
        left = max(i - l_size, 0)
        right = min(i + r_size + 1, len(lemmata))
        window = lemmata[left:right]

        if w2 in window:
            cooccurrences += 1

    return cooccurrences

def count_ngram_non_collocations(x, w1, w2, l_size: int = 1, r_size: int = 1):
    lemmata = [t.lemma_ for t in x]

    indexes = [i for i, lemma in enumerate(lemmata) if lemma == w1]

    non_cooccurrences = 0

    for i in indexes:
        left = max(i - l_size, 0)
        right = min(i + r_size + 1, len(lemmata))
        window = lemmata[left:right]

        if w2 not in window:
            non_cooccurrences += 1

    return non_cooccurrences

r1 = len([t for t in df["nlp_docs"].explode() if t.lemma_ == node])
r2 = len([t for t in df["nlp_docs"].explode() if t.lemma_ != node])
c1 = len([t for t in df["nlp_docs"].explode() if t.lemma_ == collocate])
c2 = len([t for t in df["nlp_docs"].explode() if t.lemma_ != collocate])

o11 = df["nlp_docs"].apply(count_ngram_collocations, args=(node, collocate)).sum()
o12 = df["nlp_docs"].apply(count_ngram_non_collocations, args=(node, collocate)).sum()
# o12 = r1 - o11
o21 = c1 - o11

print(f"r1: {r1}, r2: {r2}, c1: {c1}, c2: {c2}, o11: {o11}, o12: {o12}, o21: {o21}")

r1: 38, r2: 311516, c1: 26932, c2: 284622, o11: 23, o12: 15, o21: 26909


For this last question, I have to admit that I was completely lost. After many countless attempts to fix the code by trial and error, I referenced the updated code in the week_04_inclass.ipynb, from which I copied and pasted the code above. (I hope this was allowed; I assumed this was "open book" like the other quizzes). My coding skills are still very, very basic, and I'm not able to produce or fully understand such complex codes yet. I thought it was better to use the class notebook and try to understand why it works rather than just give up and leave the question blank. 

I had a feeling that sum(cooccurences) was incorrect and was part of the problem because it was adding frequencies together instead of counting the collocations (how o11 was higher than r1 and the o12 was negative), but I wouldn't have been able to come up with creating an index to "enumerate" the cooccurrences on my own. And because I was so hyper-focused on trying to make small changes to the already existing code, it didn't even cross my mind that, since there was one to count the collocations, there needed to be a function to count the non-collocations as well.





